# Archaeal histone diversity: a step by step analysis.

In [1]:
import sys
from Bio import SeqIO
from Bio.SeqUtils import ProtParam
import matplotlib 
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import statistics
from statistics import stdev
from statistics import mean
from sklearn.cluster import DBSCAN
from sklearn import metrics
import numpy as np
import pandas as pd
import plotly.express as px
import plotly
import plotly.figure_factory as ff
import math
import seaborn as sns

In [2]:
def import_file(input_file):
    imported_proteins = []
    sequence = None
    seq_len = 0
    gene_count = 0

    with open(input_file, 'r') as fastafile:
        for line in fastafile:
            if '|' in line:
                continue
            if '>' in line:
                gene_count += 1
                if sequence is not None:
                    seq_len = len(sequence)
                    protein.append(sequence)
                    protein.append(seq_len)
                    sequence = None
                    imported_proteins.append(protein)
                protein = []
                protein.append(line.rstrip('\n'),)
            elif (len(line) != 0) and (sequence is not None):
                sequence += (line.rstrip('\n'))
            elif (len(line) != 0) and (sequence is None):
                sequence = (line.rstrip('\n'))
        seq_len = len(sequence)
        protein.append(sequence)
        protein.append(seq_len)
        imported_proteins.append(protein)
    return imported_proteins

In [3]:
#all_imported_proteins = import_file('archaeal_proteins_20191203.fa')

In [4]:
imported_proteins = import_file('archaeal_histones_20191211.fa')

In [5]:
imported_proteins_atts = []
excluded_proteins = 0
counter = 0
for protein in imported_proteins:
    counter += 1
    if counter % 1000 == 0:
        print(counter)
    new_protein = []
    if 'X' in protein[1]:
        excluded_proteins += 1
        continue
    if 'O' in protein[1]:
        excluded_proteins += 1
        continue
    if 'U' in protein[1]:
        excluded_proteins += 1
        continue
    if 'B' in protein[1]:
        excluded_proteins += 1
        continue
    if 'partial' in protein[0]:
        excluded_proteins += 1
        continue
    X = ProtParam.ProteinAnalysis(protein[1])
    new_protein.append(protein[0])
    new_protein.append(protein[1])
    new_protein.append(protein[2]) 
    new_protein.append(X.aromaticity())
    new_protein.append(X.isoelectric_point()) 
    new_protein.append(X.secondary_structure_fraction()[0])
    new_protein.append(X.secondary_structure_fraction()[1])
    new_protein.append(X.secondary_structure_fraction()[2])
    new_protein.append(X.instability_index())
    new_protein.append(sum(X.flexibility()) / protein[2])
    new_protein.append(X.gravy())
    imported_proteins_atts.append(new_protein)
print('Number of excluded proteins: '+str(excluded_proteins))

1000
2000
3000
Number of excluded proteins: 44


In [6]:
def standardize_data(data_list):
    standard_list = []
    list_stdev = statistics.stdev(data_list)
    list_mean = statistics.mean(data_list)
    for i in data_list:
        new_i = (i - list_mean) / list_stdev
        standard_list.append(new_i)
    return standard_list

In [7]:
names = []
lengths = []
aromaticities = []
isos = []
helices = []
turns = []
sheets = []
instabilities = []
flexibilities = []
gravies = []
for protein in imported_proteins_atts:
    names.append(protein[0])
    lengths.append(protein[2])
    aromaticities.append(protein[3])
    isos.append(protein[4])
    helices.append(protein[5])
    turns.append(protein[6])
    sheets.append(protein[7])
    instabilities.append(protein[8])
    flexibilities.append(protein[9])
    gravies.append(protein[10])
new_lengths = standardize_data(lengths)
new_aromaticities = standardize_data(aromaticities)
new_isos = standardize_data(isos)
new_helices = standardize_data(helices)
new_turns = standardize_data(turns)
new_sheets = standardize_data(sheets)
new_instabilities = standardize_data(instabilities)
new_flexibilities = standardize_data(flexibilities)
new_gravies = standardize_data(gravies)

In [8]:
X = np.c_[new_lengths, new_isos, new_helices, new_gravies]
db = DBSCAN(eps=0.5, min_samples=10).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

In [9]:
old_tab_20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),  
         (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),  
         (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),  
         (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),  
         (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]
tab_20_pastel = []
tab_20 = []
next_list = []
counter = 0
for color in old_tab_20:
    if counter % 2 == 1:
        tab_20_pastel.append('rgb'+str(color))
    else:
        tab_20.append('rgb'+str(color))
    counter += 1

In [10]:
cluster_names = []
for label in labels:
    cluster_names.append('Cluster '+str(label))

In [11]:
#histone_species = []
#for name in histones_clustered['Name']:
 #   histone_species.append(name.split('[')[1].split(']')[0])
#print(len(histone_species))
#uni_histone_species = set(histone_species)
#print(len(uni_histone_species))

In [12]:
#all_species = []
#counter = 0
#total = len(all_imported_proteins)
#for name in all_imported_proteins:
 #   all_species.append(name[0].split('[')[1].split(']')[0])
  #  counter += 1
#print(len(all_species))
#uni_all_species = set(all_species)
#print(len(uni_all_species))

In [13]:
#print(len(uni_all_species))
#print(len(uni_histone_species))

In [14]:
#non_histone_species = []
#for species in uni_all_species:
 #   if species not in uni_histone_species:
  #      non_histone_species.append(species)
        
#print(len(non_histone_species))

In [15]:
coordinates = zip(new_lengths, new_isos, new_helices, new_gravies, names)
old_cluster_coords = {}
for point, label in zip(coordinates, labels):
    if label == -1:
        label = 'null'
    if label in old_cluster_coords:
        old_cluster_coords[label].append(point)
    else:
        old_cluster_coords[label] = [point]
nulls = old_cluster_coords.pop('null')
sort_list = []
[sort_list.append(i) for i in old_cluster_coords]
sort_list = sorted(sort_list)
cluster_coords = {}
cluster_coords['null'] = nulls 

for i in sort_list:
    cluster_coords[i] = old_cluster_coords[i]
print([i for i in cluster_coords])

['null', 0, 1, 2, 3, 4, 5, 6, 7]


In [16]:
center_l = []
center_i = []
center_h = []
center_g = []
for key in cluster_coords:
    cluster_l = []
    cluster_i = []
    cluster_h = []
    cluster_g = []
    for point in cluster_coords[key]:
        cluster_l.append(point[0])
        cluster_i.append(point[1])
        cluster_h.append(point[2])
        cluster_g.append(point[3])
    center_l.append(statistics.mean(cluster_l))
    center_i.append(statistics.mean(cluster_i))
    center_h.append(statistics.mean(cluster_h))
    center_g.append(statistics.mean(cluster_g))
centroids = zip(center_l, center_i, center_h, center_g)

In [17]:
def find_closest_point(centroid, points):
    distances = {}
    for point in points:
        l2 = (centroid[0] - float(point[0])) **2
        i2 = (centroid[1] - float(point[1])) **2
        h2 = (centroid[2] - float(point[2])) **2
        g2 = (centroid[3] - float(point[3])) **2
        distance = math.sqrt(l2 + i2 + h2 + g2)
        distances[distance] = [float(point[0]), float(point[1]), float(point[2]), float(point[3]), point[4]]
    closest_point = distances[min(distances.keys())]
    return closest_point

In [18]:
np_coordinates = np.c_[new_lengths, new_isos, new_helices, new_gravies, names]
closest_points = []
for centroid in centroids:
    closest_point = find_closest_point(centroid, np_coordinates)
    closest_points.append(closest_point)

In [19]:
def unstandardize_data(data_list, coords):
    unstandard_list = []
    list_stdev = statistics.stdev(data_list)
    list_mean = statistics.mean(data_list)
    for i in coords:
        new_i = (i * list_stdev) + list_mean
        unstandard_list.append(new_i)
    return unstandard_list

In [20]:
uncenter_l = unstandardize_data(lengths, center_l)
uncenter_i = unstandardize_data(isos, center_i)
uncenter_h = unstandardize_data(helices, center_h)
uncenter_g = unstandardize_data(gravies, center_g)

In [21]:
new_closest_l = []
new_closest_i = []
new_closest_h = []
new_closest_g = []
new_closest_names = []
for point in closest_points:
    new_closest_l.append(point[0])
    new_closest_i.append(point[1])
    new_closest_h.append(point[2])
    new_closest_g.append(point[3])
    new_closest_names.append(point[4])

In [22]:
closest_l = unstandardize_data(lengths, new_closest_l)
closest_i = unstandardize_data(isos, new_closest_i)
closest_h = unstandardize_data(helices, new_closest_h)
closest_g = unstandardize_data(gravies, new_closest_g)

In [26]:
histones_clustered = pd.DataFrame({'Length': lengths, 'pI': isos, 'Helical character': helices, 'Cluster' : cluster_names, 'Name': names})
histones_clustered.sort_values('Cluster', axis=0, ascending=True, inplace=True, kind='quicksort')
fig = px.scatter_3d(histones_clustered, x='Length', y='pI', z='Helical character', color='Cluster', color_discrete_sequence=tab_20_pastel, hover_name='Name').for_each_trace(lambda t: t.update(name=t.name.replace("Cluster=","")))
fig.update_traces(marker=dict(size=3))
fig.update_layout(title={'text':"Diversity of archaeal histones"}, legend= {'itemsizing': 'constant'})
fig.add_scatter3d(x=uncenter_l[1:], y=uncenter_i[1:], z=uncenter_h[1:], mode='markers', name='Centroids', marker=dict(color='#444444', size=3))
fig.add_scatter3d(x=closest_l[1:], y=closest_i[1:], z=closest_h[1:], hovertext=new_closest_names[1:], hoverinfo=('text+x+y+z'), marker=dict(color=tab_20[1:], size=7), mode='markers', name='Closest points')

#plotly.offline.plot(fig, filename='archaeal_histone_diversity_plot.html')

In [25]:
co_clusters_df = pd.DataFrame({})
for query_cluster in cluster_coords:
    if query_cluster == 'null':
        continue
    queries = []
    scores = []
    for protein in cluster_coords[query_cluster]:
        query = protein[4].split('[')[-1].replace(']', '')
        if query == 'archaeon':
            continue
        queries.append(query)
        
    for domain_cluster in cluster_coords:
        if domain_cluster == 'null':
            continue
        domains = []
        co_proteins = 0
        for domain_protein in cluster_coords[domain_cluster]:
            domain = domain_protein[4].split('[')[-1].replace(']', '')
            if query == 'archaeon':
                continue
            domains.append(domain)
        for query_protein in queries: 
            if query_protein in domains:
                co_proteins += 1

        scores.append(co_proteins / len(queries))
    co_clusters_df.insert(len(co_clusters_df.columns), query_cluster, scores, True) 
        
fig2 = px.imshow(co_clusters_df)
fig2.update_layout(
    title='Probability protein in cluster x has "brother" in cluster y',
    xaxis_title='Cluster x',
    yaxis_title='Cluster y')
fig2.show()

In [44]:
import ipywidgets as wg
from ipywidgets import interact, interact_manual

In [98]:
def cluster_and_plot(e, n, text):
    X = np.c_[new_lengths, new_isos, new_helices, new_gravies]
    db = DBSCAN(eps=e, min_samples=n).fit(X)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_
    cluster_names = []
    for label in labels:
        cluster_names.append('Cluster '+str(label))
    histones_clustered = pd.DataFrame({'Length': lengths, 'pI': isos, 'Helical character': helices, 'Cluster' : cluster_names, 'Name': names})
    histones_clustered.sort_values('Cluster', axis=0, ascending=True, inplace=True, kind='quicksort')
    fig = px.scatter_3d(histones_clustered, x='Length', y='pI', z='Helical character', color='Cluster', color_discrete_sequence=tab_20_pastel, hover_name='Name').for_each_trace(lambda t: t.update(name=t.name.replace("Cluster=","")))
    fig.update_traces(marker=dict(size=3))
    fig.update_layout(title={'text':"Diversity of archaeal histones"}, legend= {'itemsizing': 'constant'})
    
    key_search = histones_clustered[histones_clustered.Name.str.contains(text, case=False)]
    fig.add_scatter3d(x=key_search['Length'], y=key_search['pI'], z=key_search['Helical character'], hovertext=key_search['Name'], hoverinfo=('text+x+y+z'), mode='markers', name='Query', marker=dict(size=3, color='black'))
    return fig

In [99]:
e_slide = wg.FloatSlider(value=.5, min=0.05, max=2, step=0.05)
n_slide = wg.IntSlider(value=10, min=1, max=30)
text_search = wg.Text(value='Elepant', placeholder='Search protein or organism')
wg.interact(cluster_and_plot, e=e_slide, n=n_slide, text=text_search)

interactive(children=(FloatSlider(value=0.5, description='e', max=2.0, min=0.05, step=0.05), IntSlider(value=1…

<function __main__.cluster_and_plot(e, n, text)>